In [53]:
import requests
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
import unicodedata
from tqdm.notebook import tqdm

In [69]:
### 한글 숫자 필터링 함수
def test(s):
  hangul = re.compile('[^ ㄱ-ㅎㅏ-ㅣ가-힣0-9+]') # 한글과 띄어쓰기를 제외한 모든 글자 # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') # 위와 동일 
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return (result)

In [217]:
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
age1_list = []
age2_list = []
age3_list = []
age4_list = []
age5_list = []
man_list = []
woman_list = []
star2_list = []
story_list =[]
gamdog_list =[]
agerange_list = []
date='20210929'    ## 의미없는 부분인 것 같다.
for page in tqdm(range(1,41)):
  search_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&'
  url = f'{search_url}&date={date}&page={page}'
  req = requests.get(url)
  html = req.text
  soup =BeautifulSoup(html, 'html.parser')
  for i in range(1,56):
    if i == 1 or i == 12 or i == 23 or i == 34 or i == 45:
      pass
    else:
      tits = soup.select_one(f'#old_content > table > tbody > tr:nth-of-type({i}) > td.title > div > a')
      tits['href']
      new_name = tits['href']     # 영화 사이트 주소 가져오기
      new_url = f'https://movie.naver.com/{new_name}'
      new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
      new_req = requests.get(new_url)
      new_html = new_req.text
      s =BeautifulSoup(new_html, 'html.parser')
      
      title = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
      try:
        title = title.text     ### 영화 제목
      except:
        title = None
      gamdog = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-of-type(2)')
      try:
        gamdog = gamdog.text     ### 영화 감독
      except:
        gamdog = None
      ### 연령
      agerange = s.select_one('.info_spec')
      try:
        if '국내' in test(agerange.text.split('\n\n')[3]):
          y = test(agerange.text.split('\n\n')[3])
        else:
          y = test(agerange.text.split('\n\n')[4])
        y1 = y.find('국내')
        y2 = y.find('가')
        agerange = y[y1+2:y2+1]
      except:
        agerange = None

      inf_m = s.select_one("#netizen_point_graph > div > strong")
      try:
        inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
      except:
        inf = None

      star = s.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
      try:
        star = star.text       ### 관람객 평점
      except:
        star = None

      star2 = s.select_one("#pointNetizenPersentBasic")
      try:
        star2 = round(float(test2(str(star2)).split()[1])/10,2)       ### 네티즌 평점
      except:
        star2 = None

      date = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
      try:
        date = date.text
      except:
        date = None

      try:
        m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
      except:
        m = [None, None, None, None]

      try:
        jangru = test(m[1])
      except:
        jangru = None

      try:
        country = test(m[2])
      except:
        country = None

      try:
        time = test(m[3])
      except:
        time = None

      try:
        date = test(m[4])
      except:
        date = None

      age1 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(1) > strong.graph_percent")
      try:
        age1 = age1.text
      except:
        age1 = 'x'

      age2 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(2) > strong.graph_percent")
      try:
        age2 = age2.text
      except:
        age2 = 'x'

      age3 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(3) > strong.graph_percent")
      try:
        age3 = age3.text
      except:
        age3 = 'x' 

      age4 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(4) > strong.graph_percent")
      try:
        age4 = age4.text
      except:
        age4 = 'x'   

      age5 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(5) > strong.graph_percent")
      try:
        age5 = age5.text
      except:
        age5 = 'x'

      man = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(1) > div > strong.graph_point')
      try:
        man =man.text
      except:
        man = 'x'

      woman = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(2) > div > strong.graph_point')
      try:
        woman = woman.text
      except:
        woman = 'x'
      title_list.append(title)
      star2_list.append(star2)
      age1_list.append(age1)
      age2_list.append(age2)
      age3_list.append(age3)
      age4_list.append(age4)
      age5_list.append(age5)
      man_list.append(man)
      woman_list.append(woman)
      star_list.append(star)
      jangru_list.append(jangru)
      country_list.append(country)
      time_list.append(time)
      date_list.append(date)
      info_list.append(inf)
      gamdog_list.append(gamdog)
      agerange_list.append(agerange)


  0%|          | 0/40 [00:00<?, ?it/s]

In [219]:
data = {'영화제목':title_list,
        '감독':gamdog_list,
        '등급':agerange_list,
        '관람객평점':star_list,
        '네티즌평점':star2_list,
        '장르':jangru_list,
        '국적':country_list,
        '영화시간':time_list,
        '개봉일':date_list,
        '추천':info_list,
        '10대 비율':age1_list,
        '20대 비율':age2_list,
        '30대 비율':age3_list,
        '40대 비율':age4_list,
        '50대 비율':age5_list,
        '남성 만족도':man_list,
        '여성 만족도':woman_list}
df = pd.DataFrame(data=data)
df.tail(50)

,영화제목,감독,등급,관람객평점,네티즌평점,장르,국적,영화시간,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
1950,링컨 차를 타는 변호사,브래드 퍼맨,15세 관람가,관람객 평점 없음,None,범죄 드라마 스릴러,미국,118분,20110616 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,x,x,x,x,x,8.15,8.36
1951,스카우트,김현석,12세 관람가,관람객 평점 없음,None,코미디 멜로로맨스,한국,94분,20071114 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,x,x,x,x,x,8.17,8.26
1952,신비한 동물사전,데이빗 예이츠,12세 관람가,관람객 평점 8.69점,None,판타지 모험,영국 미국,132분,20161116 개봉,이 영화는 20대 여성이 좋아하는 영상미가 뛰어난 영화입니다.,6%,66%,15%,10%,2%,6.92,7.98
1953,바람의 검심 : 교토 대화재편,오오토모 케이시,15세 관람가,관람객 평점 9.44점,None,액션 모험,일본,139분,20150226 개봉,이 영화는 30대 남성이 좋아하는 연출이 뛰어난 영화입니다.,6%,38%,50%,6%,0%,8.11,9.04
1954,슬리피 할로우,팀 버튼,청소년 관람불가,관람객 평점 없음,None,판타지 스릴러 액션 공포,미국,111분,20000129 개봉,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,x,x,x,x,x,7.92,8.56
1955,하트 오브 더 씨,론 하워드,12세 관람가,관람객 평점 8.16점,None,액션 모험 드라마,미국,121분,20151203 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,4%,50%,25%,16%,5%,8.06,8.82
1956,황시,로저 스포티스우드,12세 관람가,관람객 평점 없음,None,전쟁 드라마,오스트레일리아 중국 독일,124분,20080918 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,x,x,x,x,x,7.85,8.66
1957,맨 프롬 UNCLE,가이 리치,12세 관람가,관람객 평점 8.09점,None,액션 모험 코미디,미국,116분,20151028 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,4%,59%,24%,9%,3%,7.88,9.02
1958,인디펜던스 데이,롤랜드 에머리히,전체 관람가,관람객 평점 없음,None,액션,미국,145분,19960727 개봉,이 영화는 10대 남성이 좋아하는 영상미가 뛰어난 영화입니다.,x,x,x,x,x,8.14,8.40
1959,아델라인 : 멈춰진 시간,리 톨랜드 크리거,12세 관람가,관람객 평점 7.70점,None,드라마 멜로로맨스,미국,112분,20151015 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,0%,56%,24%,9%,11%,8.16,8.62


In [220]:
df.isna().sum()

영화제목        25
감독          25
등급          27
관람객평점       26
네티즌평점     2000
장르          25
국적          25
영화시간        25
개봉일         36
추천          26
10대 비율       0
20대 비율       0
30대 비율       0
40대 비율       0
50대 비율       0
남성 만족도       0
여성 만족도       0
dtype: int64

In [223]:
df.to_csv('네이버_랭킹1~2000.csv')

In [140]:
man = s.select_one('.info_spec')#content > div.article > div.mv_info_area > div.mv_info > dl > dt.step4 > em
man2 = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')

In [150]:
for i in range(3,5)
  if '국내' in test(man.text.split('\n\n')[i])


True

In [210]:
##### 얻어
agerange = s.select_one('.info_spec')
#if '국내' in test(agerange.text.split('\n\n')[3]):
#  y = test(agerange.text.split('\n\n')[3])
#else:
#  y = test(agerange.text.split('\n\n')[4])
#y1 = y.find('국내')
#y2 = y.find('관람가')
#agerange = y[y1+2:y2+3]
agerange

<p class="info_spec">
<span>
<a href="/movie/sdb/browsing/bmovie.naver?genre=1">드라마</a>, 
				
			
				
				<a href="/movie/sdb/browsing/bmovie.naver?genre=11">코미디</a>
</span>
<span>
<a href="/movie/sdb/browsing/bmovie.naver?nation=US">미국</a>
</span>
<span>157분 </span>
<span>
<a href="/movie/sdb/browsing/bmovie.naver?open=1993">1993</a>
<a href="/movie/sdb/browsing/bmovie.naver?open=19930320">.03.20</a> 개봉
					
					
				
				</span>
<span>
					[국내]
						
							<a href="/movie/sdb/browsing/bmovie.naver?grade=1001003">15세 관람가</a>
							
						
					

					[해외]
						
							<a href="/movie/sdb/browsing/bmovie.naver?grade=1011004">R</a>
<a class="help" href="#" onclick="popupMovieGrade(); return false;" target="_blank" title="도움말"><em>도움말</em></a><!-- N=a:ifo.gradeguide -->
</span>
</p>

In [213]:
#### 김한용님이 설명해 주신 새로운 방법
### class index의 불규칙성 해결
dts = s.select('.info_spec dt')
dds = s.select('.info_spec dd')
info_d = {}
for dt,dd in zip(dts,dds):
  info_d[dt.text]=test(dd.text)
agerange = info_d

In [216]:
info_d

{'감독': '마틴 브레스트',
 '개요()': '드라마 코미디미국157분  19930320 개봉',
 '등급': '국내 15세 관람가 해외 도움말',
 '출연': '알 파치노프랭크 슬레이드더보기'}